In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
from torch import nn, optim, cuda
from torch.utils import data
from torchvision import datasets, transforms
import torch.nn.functional as F
import time

# Training settings
batch_size = 64
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.mp = nn.MaxPool2d(2)
        self.fc = nn.Linear(320, 10)

    def forward(self, x):
        in_size = x.size(0)
        x = F.relu(self.mp(self.conv1(x)))
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc(x)
        return F.log_softmax(x)


model = Net()
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).item()
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')


if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time()
        train(epoch)
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Training time: {m:.0f}m {s:.0f}s')
        test()
        m, s = divmod(time.time() - epoch_start, 60)
        print(f'Testing time: {m:.0f}m {s:.0f}s')

    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')



Train Epoch: 9 | Batch Status: 3200/60000 (5%) | Loss: 0.032089
Train Epoch: 9 | Batch Status: 3840/60000 (6%) | Loss: 0.034909
Train Epoch: 9 | Batch Status: 4480/60000 (7%) | Loss: 0.009963
Train Epoch: 9 | Batch Status: 5120/60000 (9%) | Loss: 0.114862
Train Epoch: 9 | Batch Status: 5760/60000 (10%) | Loss: 0.009309
Train Epoch: 9 | Batch Status: 6400/60000 (11%) | Loss: 0.074461
Train Epoch: 9 | Batch Status: 7040/60000 (12%) | Loss: 0.131663
Train Epoch: 9 | Batch Status: 7680/60000 (13%) | Loss: 0.030535
Train Epoch: 9 | Batch Status: 8320/60000 (14%) | Loss: 0.039867
Train Epoch: 9 | Batch Status: 8960/60000 (15%) | Loss: 0.056245
Train Epoch: 9 | Batch Status: 9600/60000 (16%) | Loss: 0.127311
Train Epoch: 9 | Batch Status: 10240/60000 (17%) | Loss: 0.086779
Train Epoch: 9 | Batch Status: 10880/60000 (18%) | Loss: 0.035014
Train Epoch: 9 | Batch Status: 11520/60000 (19%) | Loss: 0.055682
Train Epoch: 9 | Batch Status: 12160/60000 (20%) | Loss: 0.073267
Train Epoch: 9 | Batch St